<a href="https://colab.research.google.com/github/magland/spikeforest_batch_run/blob/master/notebooks/assemble_website_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assemble website data

This notebook assembles the data for the website.


This is the info to Liz on 11/16/18:


Here's the data for the website:
```
kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='studies'
    )
)

kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='recordings'
    )
)

kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='true_units'
    )
)

kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='sorters'
    )
)
```

Temporary locations for convenience (WARNING: don't use these because they are not up to date):

* studies: http://132.249.245.245:24351/7317cea8265b/download/3/84/3840c06ec9f048e29b0f8cfdab5eb9208a2df8a9
* recordings: http://132.249.245.245:24351/7317cea8265b/download/d/81/d81a8498fa880d1e6f7671f0d5b23255ec86ec52
* units: http://132.249.245.245:24351/7317cea8265b/download/3/1c/31cbe8f7b423301665f4c48f11a689a7567f33c7
* sorters: http://132.249.245.245:24351/7317cea8265b/download/8/e5/8e5e7cb801475b2a171228ad50fc4d6ec9bb2689


These are each tables, and the records of the tables are like this:

```
**** Study:
Study:
{
    "name": "bionet_drift",
    "study_set": "bionet",
    "directory": "kbucket://15734439d8cf/groundtruth/bionet/bionet_drift",
    "description": "...",
    "sorters": [
        "MountainSort4-thr3",
        "IronClust-drift"
    ]
}

**** Recording:
{
    "name": "001_synth",
    "study": "magland_synth_noise10_K10_C4",
    "directory": "kbucket://15734439d8cf/groundtruth/magland_synth/datasets_noise10_K10_C4/001_synth",
    "description": "One of the recordings in the magland_synth_noise10_K10_C4 study",
    "computed_info": {
        "samplerate": 30000.0,
        "num_channels": 4,
        "duration_sec": 600.0
    },
    "plots": {
        "timeseries": "sha1://cd0958b1f58faed6764e703deb3b70e59b2f1d27/timeseries.jpg",
        "waveforms_true": "sha1://70925b4a481ba6119623a6a487429fe898e18e54/waveforms.jpg"
    },
    "firings_true": "kbucket://15734439d8cf/groundtruth/magland_synth/datasets_noise10_K10_C4/001_synth/firings_true.mda",
    "true_units_info": "sha1://eec30f7c5e1d97ff6078110427e2f397c117790c/true_units_info.json"
}

**** Unit:
{
    "unit_id": 1,
    "snr": 25.396783859187707,
    "peak_channel": 0,
    "num_events": 1398,
    "firing_rate": 2.33,
    "study": "magland_synth_noise10_K10_C4",
    "recording": "001_synth",
    "sorting_results": {
        "MountainSort4-thr3": {
            "Unit ID": 2,
            "Accuracy": "0.99",
            "Best unit": 6,
            "Matched unit": 6,
            "f.n.": "0.00",
            "f.p.": "0.01",
            "# matches": 1394
        },
        "IronClust-tetrode": {
            "Unit ID": 2,
            "Accuracy": "0.98",
            "Best unit": 7,
            "Matched unit": 7,
            "f.n.": "0.00",
            "f.p.": "0.01",
            "# matches": 1381
        },
        "SpykingCircus": {
            "Unit ID": 2,
            "Accuracy": "0.99",
            "Best unit": 8,
            "Matched unit": 8,
            "f.n.": "0.00",
            "f.p.": "0.01",
            "# matches": 1384
        }
    }
}

**** Sorter:
{
    "name": "MountainSort4-thr3",
    "processor_name": "MountainSort4",
    "params": {
        "detect_sign": -1,
        "adjacency_radius": 100,
        "detect_threshold": 3
    }
}
```

Right now there are:
```
3 sorters
8 studies
80 recordings
1200 true units
3360 sorted units
```

Note that num_true_units*num_sorters should equal num_sorted_units.

However 3360 <> 1200*3. So that means some units, for some sorters, do not have an accuracy. For those, you should assign 0.



In [0]:
# Only run this cell if you are running this on a hosted runtime that does not have these packages installed
# Consider connecting to a local runtime
%%capture
!pip install spikeforest

In [0]:
from kbucket import client as kb
import spikeforest as sf
import spikeinterface as si
import json
import numpy as np

In [34]:
## Configure read/write access to kbucket
sf.kbucketConfigRemote(name='spikeforest1-readwrite',ask_password=True)

Enter password: ··········
Pairio user set to spikeforest. Test succeeded.


In [0]:
recording_collection_names=[
    'spikeforest_magland_synth_recordings',
    'spikeforest_bionet_recordings'
]

batch_names=[
    'summarize_recordings_magland_synth',
    'summarize_recordings_bionet',
    'ms4_magland_synth_dev4',
    'irc_magland_synth_dev4',
    'sc_magland_synth_dev4',
    'ms4_bionet',
    'irc_bionet',
    'sc_bionet'
]

In [41]:
## Load the SpikeForest data
SF=sf.SFData()
for rcname in recording_collection_names:
  print(rcname)
  SF.loadRecordings(key=dict(name=rcname))
for bname in batch_names:
  print(bname)
  SF.loadProcessingBatch(key=dict(batch_name=bname,name='job_results'))

spikeforest_magland_synth_recordings
spikeforest_bionet_recordings
summarize_recordings_magland_synth
Loaded 0 sorting results and 80 recording summary results
summarize_recordings_bionet
Loaded 0 sorting results and 36 recording summary results
ms4_magland_synth_dev4
Loaded 80 sorting results and 0 recording summary results
irc_magland_synth_dev4
Loaded 80 sorting results and 0 recording summary results
sc_magland_synth_dev4
Loaded 80 sorting results and 0 recording summary results
ms4_bionet
Loaded 36 sorting results and 0 recording summary results
irc_bionet
Loaded 36 sorting results and 0 recording summary results
sc_bionet
Loaded 36 sorting results and 0 recording summary results


In [0]:
def format_time(time_sec):
  return '{} min'.format(time_sec/60)

def format_time2(time1_sec,time2_sec):
  return '{} - {} min'.format(time1_sec/60,time2_sec/60)

def format_time_range(duration_sec_list):
  min0=np.min(duration_sec_list)
  max0=np.max(duration_sec_list)
  if min0==max0:
    return format_time(min0)
  else:
    return format_time2(min0,max0)

def format_int_range(list):
  min0=np.min(list)
  max0=np.max(list)
  if min0==max0:
    return '{}'.format(min0)
  else:
    return '{} - {}'.format(min0, max0)
  
def format_freq(freq_hz):
  return '{} kHz'.format(freq_hz/1000)

def format_time2(freq1_hz,freq2_hz):
  return '{} - {} kHz'.format(freq1_hz/1000,freq2_hz/1000)
  
def format_freq_range(freq_hz_list):
  min0=np.min(freq_hz_list)
  max0=np.max(freq_hz_list)
  if min0==max0:
    return format_freq(min0)
  else:
    return format_freq2(min0,max0)

def compute_study_info(study):
  rnames=study.recordingNames()
  summary_objects=[study.recording(rname).getSummaryObject() for rname in rnames]
  
  duration_sec_list=[float(obj['computed_info']['duration_sec']) for obj in summary_objects]
  num_channels_list=[int(obj['computed_info']['num_channels']) for obj in summary_objects]
  samplerate_list=[int(obj['computed_info']['samplerate']) for obj in summary_objects]
  num_true_units_list=[len(study.recording(rname).trueUnitsInfo(format='json')) for rname in rnames]
  
  info=dict(
      duration=format_time_range(duration_sec_list),
      num_channels=format_int_range(num_channels_list),
      samplerate=format_freq_range(samplerate_list),
      num_ground_truth_units=format_int_range(num_true_units_list)
  )
  
  return info

In [72]:
## Load the studies
print('Loading studies')
all_studies=[]
studies_by_name=dict()
for sname in SF.studyNames():
  print(sname)
  study=dict(name=sname)
  study['sorters']=[] # initialize
  study['info']=compute_study_info(SF.study(sname))
  studies_by_name[sname]=study
  all_studies.append(study)

Loading studies
magland_synth_noise10_K10_C4
magland_synth_noise10_K10_C8
magland_synth_noise10_K20_C4
magland_synth_noise10_K20_C8
magland_synth_noise20_K10_C4
magland_synth_noise20_K10_C8
magland_synth_noise20_K20_C4
magland_synth_noise20_K20_C8
bionet_drift
bionet_shuffle
bionet_static


In [73]:
## Load the recordings
print('Loading recordings')
all_recordings=[]
for sname in SF.studyNames():
  SS=SF.study(sname)
  for rname in SS.recordingNames():
    RR=SS.recording(rname)
    recording=RR.getObject()
    all_recordings.append(recording)

Loading recordings


In [74]:
## Load the units
all_true_units=[]
unit_lookup=dict()
for sname in SF.studyNames():
  SS=SF.study(sname)
  for rname in SS.recordingNames():
    RR=SS.recording(rname)
    true_units_info=RR.trueUnitsInfo(format='json')
    for unit in true_units_info:
      unit['study']=sname
      unit['recording']=rname
      unit['sorting_results']=dict()
      all_true_units.append(unit)
      code=sname+'---'+rname+'---'+str(unit['unit_id'])
      unit_lookup[code]=unit
print('Found {} true units'.format(len(all_true_units)))

Found 4321 true units


In [75]:
## Load the sorting results
print('Loading sorting results')
count=0
sorters_by_name=dict()
for sname in SF.studyNames():
  SS=SF.study(sname)
  for rname in SS.recordingNames():
    RR=SS.recording(rname)
    for srname in RR.sortingResultNames():
      SR=RR.sortingResult(srname)
      result=SR.getObject()
      if not SR.sorterName() in studies_by_name[sname]['sorters']:
        studies_by_name[sname]['sorters'].append(SR.sorterName())
      obj=SR.comparisonWithTruth(format='json')
      sorter=dict(
          name=result['sorter_name'],
          processor_name=result['sorting_processor_name'],
          processor_version=result['sorting_processor_version'],
          params=result['sorting_params']
      )
      sorters_by_name[SR.sorterName()]=sorter
      for unit_id in obj:
        unit=obj[unit_id]
        code=sname+'---'+rname+'---'+str(unit_id)
        if code in unit_lookup:
          unit_lookup[code]['sorting_results'][SR.sorterName()]=unit
          count=count+1
print('Loaded {} sorted units'.format(count))


Loading sorting results
Loaded 4632 sorted units


In [76]:
target='spikeforest_website_dev'

all_sorters=[]
for sname in sorters_by_name:
  all_sorters.append(sorters_by_name[sname])
print('Found {} sorters'.format(len(all_sorters)))

print('Saving {} studies'.format(len(all_studies)))
kb.saveObject(
    key=dict(
        target=target,
        name='studies'
    ),
    object=dict(
        studies=all_studies
    )
)
    
print('Saving {} recordings'.format(len(all_recordings)))
kb.saveObject(
    key=dict(
        target=target,
        name='recordings'
    ),
    object=dict(
        recordings=all_recordings
    )
)


print('Saving {} true units'.format(len(all_true_units)))
kb.saveObject(
    key=dict(
        target=target,
        name='true_units'
    ),
    object=dict(
        true_units=all_true_units
    )
)

print('Saving {} sorters'.format(len(all_sorters)))
kb.saveObject(
    key=dict(
        target=target,
        name='sorters'
    ),
    object=dict(
        sorters=all_sorters
    )
)

Found 4 sorters
Saving 11 studies
Already on server.
Saving 116 recordings
Already on server (*)
Saving 4321 true units
Already on server.
Saving 4 sorters
Already on server (*)


In [77]:
print('Study:')
obj=kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='studies'
    )
)
print(json.dumps(obj['studies'][0],indent=4))

print('Recording:')
obj=kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='recordings'
    )
)
print(json.dumps(obj['recordings'][0],indent=4))


print('Unit:')
obj=kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='true_units'
    )
)
print(json.dumps(obj['true_units'][0],indent=4))

print('Sorter:')
obj=kb.loadObject(
    key=dict(
        target='spikeforest_website_dev',
        name='sorters'
    )
)
print(json.dumps(obj['sorters'][0],indent=4))

Study:
{
    "name": "magland_synth_noise10_K10_C4",
    "sorters": [
        "MountainSort4-thr3",
        "IronClust-tetrode",
        "SpykingCircus"
    ],
    "info": {
        "duration": "10.0 min",
        "num_channels": "4",
        "samplerate": "30.0 kHz",
        "num_ground_truth_units": "10"
    }
}
Recording:
{
    "name": "001_synth",
    "study": "magland_synth_noise10_K10_C4",
    "directory": "kbucket://15734439d8cf/groundtruth/magland_synth/datasets_noise10_K10_C4/001_synth",
    "description": "One of the recordings in the magland_synth_noise10_K10_C4 study"
}
Unit:
{
    "unit_id": 1,
    "snr": 25.396783859187707,
    "peak_channel": 0,
    "num_events": 1398,
    "firing_rate": 2.33,
    "study": "magland_synth_noise10_K10_C4",
    "recording": "001_synth",
    "sorting_results": {
        "MountainSort4-thr3": {
            "Unit ID": 2,
            "Accuracy": "0.99",
            "Best unit": 6,
            "Matched unit": 6,
            "f.n.": "0.00",
     

In [78]:
print('studies: '+kb.findFile(key=dict(
    target='spikeforest_website',
    name='studies'
),local=False,remote=True))

print('recordings: '+kb.findFile(key=dict(
    target='spikeforest_website',
    name='recordings'
),local=False,remote=True))

print('units: '+kb.findFile(key=dict(
    target='spikeforest_website',
    name='true_units'
),local=False,remote=True))

print('sorters: '+kb.findFile(key=dict(
    target='spikeforest_website',
    name='sorters'
),local=False,remote=True))

studies: http://132.249.245.245:24351/7317cea8265b/download/a/1c/a1c13733b50a1bdf830be314b24711662d89e0e1
recordings: http://132.249.245.245:24351/7317cea8265b/download/b/eb/beb63dd5b4dda4ba49eb08c3af7c0319a07b8f38
units: http://132.249.245.245:24351/7317cea8265b/download/f/5b/f5bb658dc4e118a530c749211a752f7bfb1093e8
sorters: http://132.249.245.245:24351/7317cea8265b/download/f/b0/fb04a740198198d8f6fe25e64e73870c55558546


In [0]:
## This was the old method

all_sorting_results=[]
all_summarize_recording_results=[]
for bname in batch_names:
  print('Loading '+bname)
  obj=kb.loadObject(key=dict(batch_name=bname,name='job_results'))
  job_results=obj['job_results']
  for res in job_results:
    if res['job']['command']=='sort_recording':
      all_sorting_results.append(res)
    elif res['job']['command']=='summarize_recording':
      if 'true_units_info' in res['result']:
        all_summarize_recording_results.append(res)
      else:
        print('WARNING: no field, true_units_info, skipping.')
        display(res)

import json
def load_json(fname):
  fname=kb.realizeFile(fname)
  with open(fname) as f:
    return json.load(f)

def use_study(name):
  if name.startswith('magland'):
    return True
  if name.startswith('bionet'):
    return True
  return False

## Load the studies
print('Loading studies')
all_studies=[]
studies_by_name=dict()
obj=kb.loadObject(key=dict(name='spikeforest_recordings'))
studies=obj['studies']
for study in studies:
  study['sorters']=[] # initialize
  studies_by_name[study['name']]=study
  if use_study(study['name']):
    all_studies.append(study)

## Load the recordings
print('Loading recordings')
all_recordings=[]
for res in all_summarize_recording_results:
  recording=res['result']
  study=recording['study']
  if use_study(recording['study']):
    all_recordings.append(recording)
    
## Load the units
all_true_units=[]
unit_lookup=dict()
print('Loading summarize recording results')
for res in all_summarize_recording_results:
  study=res['job']['recording']['study']
  recording=res['job']['recording']['name']
  if use_study(study):
    obj=load_json(res['result']['true_units_info'])
    for unit in obj:
      unit['study']=study
      unit['recording']=recording
      unit['sorting_results']=dict()
      all_true_units.append(unit)
      code=study+'---'+recording+'---'+str(unit['unit_id'])
      unit_lookup[code]=unit
  #res['result']['true_units_info_data']=obj
print('Found {} true units'.format(len(all_true_units)))

## Load the sorting results
print('Loading sorting results')
count=0
sorters_by_name=dict()
for res in all_sorting_results:
  study=res['job']['recording']['study']
  recording=res['job']['recording']['name']
  sorter=res['job']['sorter']['name']
  if not sorter in studies_by_name[study]['sorters']:
    studies_by_name[study]['sorters'].append(sorter)
  if use_study(study):
    obj=load_json(res['result']['comparison_with_truth']['json'])
    sorters_by_name[sorter]=res['job']['sorter']
    for unit_id in obj:
      unit=obj[unit_id]
      code=study+'---'+recording+'---'+str(unit_id)
      if code in unit_lookup:
        unit_lookup[code]['sorting_results'][sorter]=unit
        count=count+1
print('Loaded {} sorted units'.format(count))

all_sorters=[]
for sname in sorters_by_name:
  all_sorters.append(sorters_by_name[sname])
print('Found {} sorters'.format(len(all_sorters)))

print('Saving {} studies'.format(len(all_studies)))
kb.saveObject(
    key=dict(
        target='spikeforest_website',
        name='studies'
    ),
    object=dict(
        studies=all_studies
    )
)
    
print('Saving {} recordings'.format(len(all_recordings)))
kb.saveObject(
    key=dict(
        target='spikeforest_website',
        name='recordings'
    ),
    object=dict(
        recordings=all_recordings
    )
)


print('Saving units')
kb.saveObject(
    key=dict(
        target='spikeforest_website',
        name='true_units'
    ),
    object=dict(
        true_units=all_true_units
    )
)

print('Saving sorters')
kb.saveObject(
    key=dict(
        target='spikeforest_website',
        name='sorters'
    ),
    object=dict(
        sorters=all_sorters
    )
)

  
#print('Loading summarize recording results')
#for res in all_summarize_recording_results:
#  obj=load_json(res['result']['true_units_info'])
#  res['result']['true_units_info_data']=obj

Loading studies
Loading recordings
Loading summarize recording results
Found 26688 true units
Loading sorting results
Loaded 12615 sorted units
Found 4 sorters
Saving 11 studies
Saving 116 recordings
Already on server.
Saving units
Saving sorters
Already on server.
